# OUTLINE

- #### The Vision of E2EAI (End-to-End AI)
- #### Components of E2EAI
- #### The Julia AI/ML Solution Use-case
- #### The Future

# The Vision of E2EAI

- yaml description -> workflow transformer -> workflow orchestrator -> workflow management

# Components of E2EAI

- EU project SUNRISE 

## Infrastucture Creation

## Workflow AI as a Service

# The Julia AI/ML Solution Use-case

- show background of automlpipeline and tmsl
- discuss the disadvantage of manual workflow
- show the workflow with zero touch coding using e2eai

In [3]:
using AutoMLPipeline;

In [5]:
1+1

2

In [4]:
using TSML;

# Zero Coding Using Workflow Template
<center><img src="./template.png" width="1000"/></center>

## Explicit ML Pipeline

<center><img src="./mlpipeline.png" width="500"/></center>

## Optimal Pipeline Discovery by AutoML
<center><img src="./lowcode.png" width="500"/></center>